In [1]:
# change tabula to tabula_middle_padding to test middle padding method
from tabula_middle_padding import Tabula 
import pandas as pd
import torch

In [2]:
data = pd.read_csv("Real_Datasets/air_quality_cleaned.csv")
data = data.drop(['Date', 'Time'], axis=1)

In [3]:
#categorical_columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country', 'income']
model = Tabula(llm='distilgpt2', experiment_dir = "air-quality_training", batch_size=32, epochs=128)

## In following block, it is important to indicate "conditional_col = data.columns[0]". Otherwise, the generation will use target column as conditional column, which will result in error!

In [4]:
model.fit(data, conditional_col = data.columns[0])

/home/svenlag/tabula_test/Tabula/tabula_middle_padding/tabula.py:130: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `TabulaTrainer.__init__`. Use `processing_class` instead.
  tabula_trainer = TabulaTrainer(self.model, training_args, train_dataset=tabula_ds, tokenizer=self.tokenizer,
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
500,1.899400
1000,1.440100
1500,1.358200
2000,1.301900
2500,1.249200
3000,1.189900
3500,1.112400
4000,1.022900
4500,0.931400
5000,0.835600


In [5]:
import torch
torch.save(model.model.state_dict(), "air-quality_training/model_128epoch_middle_padding.pt")

In [5]:
torch.save(model, "air-quality_training/model_128epoch_middle_padding_FULL.pt")

In [3]:
# Specify weights_only=False to allow full unpickling
model = torch.load(
    "air-quality_training/model_128epoch_middle_padding_FULL.pt",
    map_location=torch.device("cuda"),      # or "cpu" as needed
    weights_only=False
)
model.model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [4]:
for i in range(9, 31):
    synthetic_data = model.sample(n_samples=6941)
    synthetic_data.to_csv(f"air-quality_sd-sets/air-quality_128epoch_middle_padding_{i}.csv", index=False)

  0%|          | 0/6941 [00:00<?, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
6994it [00:34, 203.23it/s]                          
6994it [00:44, 156.00it/s]                          
6998it [01:17, 90.65it/s]                           
6996it [01:22, 84.41it/s]                           
6996it [01:25, 81.92it/s]                           
6994it [01:25, 81.76it/s]                           
6996it [01:22, 85.07it/s]                           
6996it [01:33, 74.78it/s]                          
6998it [01:27, 79.67it/s]                          
6997it [01:23, 83.42it/s]                          
6997it [01:25, 81.36it/s]                          
6999it [01:27, 79.85it/s]                          
6995it [01:25, 82.19it/s]                          
6997it [01:29, 77.91it/s]                       

In [6]:
synthetic_data = model.sample(n_samples=6941)
synthetic_data.to_csv("air-quality_128epoch_middle_padding.csv", index=False)

  0%|          | 0/6941 [00:00<?, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
6971it [00:43, 158.58it/s]                          


In [7]:
from sdmetrics.reports.single_table import QualityReport

synthetic_data = synthetic_data[data.columns.to_list()]

metadata = {
    "columns": {
        "CO(GT)": {"sdtype": "numerical"},
        "PT08.S1(CO)": {"sdtype": "numerical"},
        "C6H6(GT)": {"sdtype": "numerical"},
        "PT08.S2(NMHC)": {"sdtype": "numerical"},
        "NOx(GT)": {"sdtype": "numerical"},
        "PT08.S3(NOx)": {"sdtype": "numerical"},
        "NO2(GT)": {"sdtype": "numerical"},
        "PT08.S4(NO2)": {"sdtype": "numerical"},
        "PT08.S5(O3)": {"sdtype": "numerical"},
        "T": {"sdtype": "numerical"},
        "RH": {"sdtype": "numerical"},
        "AH": {"sdtype": "numerical"}
    }
}

report = QualityReport()
report.generate(data, synthetic_data, metadata)

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 12/12 [00:00<00:00, 346.87it/s]|
Column Shapes Score: 77.1%

(2/2) Evaluating Column Pair Trends: |██████████| 66/66 [00:00<00:00, 289.22it/s]|
Column Pair Trends Score: 77.48%

Overall Score (Average): 77.29%

